In [1]:
import pandas as pd
import numpy as np
import folium
import json
from pathlib import Path
from ipyleaflet import Map, Choropleth, GeoJSON, WidgetControl
from branca.colormap import linear
import ipywidgets as widgets
current_dir = Path.cwd()


In [2]:
##Basic Map
m = folium.Map()

#m

In [23]:
#Map centered on California
m = folium.Map(location=[37.77,-122.41], zoom_start=10)
#gemerate map
#m.save('map.html')
#m

In [59]:

geojson_path = f'{current_dir}/data/tract_boundaries.json'
with open(geojson_path) as f:
    geojson_data = json.load(f)

#Adding "id" feature to geojson file
for tract in geojson_data['features']:
    ID = tract['properties']['GEOID']
    tract['id'] = ID

#Adding tracts present in the Geojson file but not the DAC classification data as missing values
data = pd.read_csv(f"{current_dir}/cleaned_data/geo_df.csv", dtype=str)
tracts = [tract['id'] for tract in geojson_data['features']]
missing_tracts = [tract for tract in tracts if tract not in data['GEOID'].values]

#initializing datagrame for missing tracts
missing_df = pd.DataFrame()
missing_df['GEOID'] = missing_tracts
for col in data.columns[1:]:
    missing_df[col] = np.repeat(None, len(missing_tracts))

#Appending missing tracts to DAC dataframe
new_df = pd.concat([data, missing_df], ignore_index=True)

#Classifying missing tracts as "missing"
def categorize(col):
    if col == '1':
        return 'Yes'
    elif col == '0':
        return 'No'
    else: 
        return "Missing"
new_df['DAC'] = new_df['DAC'].apply(categorize)
new_df.to_csv(f"{current_dir}/cleaned_data/geo_df.csv", index=False)




for tract in geojson_data['features']:
    ID = tract['properties']['GEOID']
    tract['DAC'] = new_df.loc[new_df['GEOID'] == ID]['DAC'].values[0]
    tract['id'] = ID
    prop = {}
    count = 0
    for key, val in tract['properties'].items():
        if count <= 4:
            prop[key] = val
            count +=1
    tract['properties'] = prop

fp = f'{current_dir}/cleaned_data/tract_boundaries.json'
with open(fp, 'w') as file:
    json.dump(geojson_data, file, indent=4)

In [63]:
dac_data = pd.read_csv(f"{current_dir}/cleaned_data/default_score.csv")
dac_data

,Census Tract,County,env_exposure,env_effect,sens_pop,ses_factors,Pollution Burden,Pop Char,Pollution Burden MinMax,Pop Char MinMax,Score,Percentile,DAC
0,6019001100,Fresno,81.90625,73.124,95.030000,91.278,78.978833,93.154000,9.643207,9.663153,93.183786,100.000000,1
1,6077000700,San Joaquin,63.92500,92.456,90.353333,95.976,73.435333,93.164667,8.966354,9.664259,86.653166,99.987393,1
2,6037204920,Los Angeles,76.37375,80.272,87.933333,79.566,77.673167,83.749667,9.483787,8.687612,82.391466,99.974786,1
3,6019000700,Fresno,72.33000,58.882,97.530000,91.752,67.847333,94.641000,8.284067,9.817404,81.328026,99.962179,1
4,6019000200,Fresno,80.54875,39.380,97.423333,93.376,66.825833,95.399667,8.159343,9.896103,80.745694,99.949571,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7927,6081609700,San Mateo,24.14000,3.328,6.356667,5.994,17.202667,6.175333,2.100422,0.640586,1.345502,0.063036,0
7928,6085507302,Santa Clara,21.20625,23.588,4.450000,4.918,22.000167,4.684000,2.686190,0.485886,1.305182,0.050429,0
7929,6013351200,Contra Costa,24.24750,22.760,4.930000,3.370,23.751667,4.150000,2.900046,0.430492,1.248448,0.037821,0
7930,6081609601,San Mateo,14.71500,0.360,9.250000,8.164,9.930000,8.707000,1.212439,0.903204,1.095080,0.025214,0


In [3]:
#Load the Data
geojson_path = f'{current_dir}/data/tract_boundaries.json'
with open(geojson_path) as f:
    geojson_data = json.load(f)

def style_function(feature):
    return {
        'fillColor': '#ffaf00',  # Fill color (hex color code)
        'color': 'black',         # Border color
        'weight': 0.5,             # Border width
        'fillOpacity': 0,      # Fill opacity
    }

m = folium.Map(location=[37.77,-122.41], zoom_start=10)
folium.GeoJson(geojson_data,
               style_function=style_function).add_to(m)
#m

In [67]:
##Working code, Could alter the color scheme
#Actually, broke somewhat after changing the geo_df.csv generation process. Is likely due to either changing the 
#DAC classifications from integers to strings or the inclusion of tracts with missing
geojson_path = f'{current_dir}/data/tract_boundaries.json'
with open(geojson_path) as f:
    geojson_data = json.load(f)

data = pd.read_csv(f"{current_dir}/cleaned_data/geo_df.csv")

def style_function(feature):
    return {
        'fillColor': '#ffaf00',  # Fill color (hex color code)
        'color': 'black',         # Border color
        'weight': 0.5,             # Border width
        'fillOpacity': 0,      # Fill opacity
    }

m = folium.Map(location=[37.77,-122.41], zoom_start=6)

folium.Choropleth(
    geo_data=geojson_data,
    data=data,
    columns=['GEOID', 'DAC'],
    key_on='properties.GEOID',
    fill_color='GnBu',
    fill_opacity=0.5,
    line_opacity=0.5,
    legend_name='DAC_Designated'
).add_to(m)
m.save(f'{current_dir}/Static/html/original_map.html')


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [3]:
# Load the GeoJSON data
#geojson_path = f'{current_dir}/cleaned_data/tract_boundaries.json'
with open(geojson_path) as f:
    geojson_data = json.load(f)


#Colors for each DAC category
colors = {
    'Yes': ['#2EEC57', 0.7],
    'No': ['#EC2E42', 0.7],
    'Missing': ['#C0C0C0', 0.5]
}

# Function to style the GeoJSON layer
def style_function(feature):
    return {
        'fillColor': colors[feature['DAC']][0],  # Fill color (hex color code)
        'color': 'black',        # Border color
        'weight': 0.1,           # Border width
        'fillOpacity': colors[feature['DAC']][1]         # Fill opacity
    }


# Create a map
m = folium.Map(location=[37.77,-122.41], zoom_start=6)

# Create a GeoJSON layer
geo_json_layer = folium.GeoJson(
    data=geojson_data,
    style_function=style_function,
    #highlight_function={'fillColor': 'yellow', 'fillOpacity': 0.2},
    name='DAC Classification'
).add_to(m)

# Add the GeoJSON layer to the map
folium.LayerControl().add_to(m)


#m.save(f'{current_dir}/Static/original_map.html')

# Display the map
#m

NameError: name 'geojson_path' is not defined

In [44]:
#Adding tracts present in the Geojson file but not the DAC classification data as missing values

data = pd.read_csv(f"{current_dir}/cleaned_data/geo_df.csv", dtype=str)
data.loc[data['GEOID'] == '06085504321']
tracts = [tract['id'] for tract in geojson_data['features']]
missing_tracts = [tract for tract in tracts if tract not in data['GEOID'].values]
#missing_tracts = np.array([[tract, None, None, None, None, None, None, None, None, None, None, None] for tract in tracts])
missing_df = pd.DataFrame()
missing_df['GEOID'] = missing_tracts
for col in data.columns[1:]:
    missing_df[col] = np.repeat(None, len(missing_tracts))
new_df = pd.concat([data, missing_df], ignore_index=True)
new_df
def categorize(col):
    if col == '1':
        return 'Yes'
    elif col == '0':
        return 'No'
    else: 
        return "Missing"
new_df['DAC'] = new_df['DAC'].apply(categorize)
new_df.to_csv(f"{current_dir}/cleaned_data/geo_df.csv", index=False)

In [64]:
fp = f'{current_dir}/cleaned_data/tract_boundaries.json'
with open(fp) as f:
    geojson_data = json.load(f)

In [66]:
geojson_data['features'][0]


{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-121.875559, 37.39924],
    [-121.875412, 37.398811],
    [-121.875, 37.397665],
    [-121.874826, 37.397166],
    [-121.874611, 37.396693],
    [-121.874331, 37.396144],
    [-121.874051, 37.395627],
    [-121.873808, 37.395275],
    [-121.873314, 37.394571],
    [-121.872745, 37.39388],
    [-121.87203, 37.393108],
    [-121.871461, 37.392628],
    [-121.87036, 37.391566],
    [-121.870054, 37.39127],
    [-121.868446, 37.389771],
    [-121.866786, 37.388198],
    [-121.866184, 37.387661],
    [-121.865445, 37.387002],
    [-121.863054, 37.384864],
    [-121.862864, 37.38467],
    [-121.862772, 37.384754],
    [-121.862462, 37.38508],
    [-121.86203, 37.385528],
    [-121.861778, 37.38579],
    [-121.861533, 37.385993],
    [-121.861125, 37.386287],
    [-121.860774, 37.386531],
    [-121.860561, 37.386558],
    [-121.860266, 37.386774],
    [-121.860226, 37.386804],
    [-121.859988, 37.38699],
    [-121.85983